# Pattern Matrix Replacement

This this notebook we explore strategies to replace the ***Hit Correlation*** step of Konrad's pipeline, specifically the *patten matrix* algorithm previously used.

## Approach
I am thinking of two approaches. First, use a simple MLP to determine if two given points are *causally related* or not. To do this, the following tasks are required:
- [x] prepare the data such that each row contains the features (x,y,z,time/timeslice) of each pair of points (exluding self)
    - [ ] subtask here is to use PCA to reduce the dimensions and see if the network performs same/worse
- [ ] create labels for training set ie. *1* of related and *0* otherwise (this hinges upon the fact that we can extract labels from *mc_info* table)
- [ ] visualize the results

The other approach is to treat this as an unsupervised learning task and use clustering to determine *related* points.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../data/data.csv")
data

,pos_x,pos_y,pos_z,time,label,event_id,timeslice
0,-17.661,32.245,65.231,0.0,0,NaN,0
1,76.840,-77.173,186.931,0.0,0,NaN,0
2,-73.403,30.509,94.511,0.0,0,NaN,0
3,1.453,33.155,169.111,0.0,0,NaN,0
4,49.456,47.904,140.111,0.0,0,NaN,0
...,...,...,...,...,...,...,...
45820211,-57.230,-5.401,196.389,101502104.0,0,NaN,6766
45820212,0.724,66.341,121.789,101516467.0,0,NaN,6767
45820213,-26.436,86.737,160.131,101545421.0,0,NaN,6769
45820214,-26.931,-21.994,178.511,101581891.0,0,NaN,6772


# Creating the "Pattern Matrix" Dataset

Since this step will essentially double the width of the dataset and square it's height, we will only use timeslice 665 (timeslice with the largest hits, as per the [exploration](notebooks/exploration.ipynb) conducted previously). The algorithm to generate the dataset is as follows:
1. create an empty dataframe to hold the `result`
2.iterate over original df with the row and index
    1. duplicate original df
    2. set value of `dup` columns to that of the row
    3. concat dup and original dfs (sideways) to create pairs
    4. drop the rows where `id1` is less than `id2` to avoid repeat pairs
    5. append dup to result
    
The algorithm was tested with a small sample of 10 rows before the dataset below was created.

In [49]:
p1_col_names = {'pos_x': 'x1', 'pos_y': 'y1',
    'pos_z': 'z1', 'time': 't1',
    'label': 'l1', 'event_id': 'eid1',
    'timeslice': 'ts1', 'id':'id1'}
p2_col_names = {'pos_x': 'x2', 'pos_y': 'y2',
    'pos_z': 'z2', 'time': 't2',
    'label': 'l2', 'event_id': 'eid2',
    'timeslice': 'ts2', 'id':'id2'}

def explode(df):
    """
    Expects a dataframe which is then used to create `result`
    containing each row of `df` with all other rows. Returns
    `result` which is a dataframe.
    """
    result = pd.DataFrame()
    for id, row in df.iterrows():
        dup = df.copy()

        # may be a better way to do this
        dup['pos_x'] = row['pos_x']
        dup['pos_y'] = row['pos_y']
        dup['pos_z'] = row['pos_z']
        dup['time'] = row['time']
        dup['label'] = row['label']
        dup['event_id'] = row['event_id']
        dup['timeslice'] = row['timeslice']
        dup['id'] = row['id']
    
        dup = dup.rename(columns=p1_col_names)

        dup = pd.concat([dup, df], axis=1)

        dup = dup.rename(columns=p2_col_names)

        dup = dup[dup['id2'] > dup['id1']]

        result = pd.concat([result, dup])
    
    return result

## Profiling explore variants

Profile the `explode` and `explode2` function on a various samples to get an estimate for it's runtime and performance.

| method | n | time |
|--------|---|------|
|explode |100|1.07  |
|explode2|100||
|explode |200|2.75  |
|explode2|200||
|explode |300|5.95  |
|explode2|300||
|explode |500|19.23 |
|explode2|500||
|explode |850|83.70 |
|explode2|850||

TODO The runtime of `explore` seems to be exponential, yet to determine why this is the case...

In [60]:
import cProfile

In [65]:
timeslice = 615
df = data[data['timeslice'] == timeslice].sample(frac=0.1)
df['id'] = df.reset_index().index
df

,pos_x,pos_y,pos_z,time,label,event_id,timeslice,id
4229943,88.533,12.246,168.889,9229080.0,0,NaN,615,0
4230540,-36.721,-78.069,94.459,9230328.0,0,NaN,615,1
4230198,-8.090,87.111,37.789,9229611.0,0,NaN,615,2
4231941,2.755,-6.400,65.341,9232477.0,1,2042.0,615,3
4229393,-75.096,101.731,187.041,9227891.0,0,NaN,615,4
...,...,...,...,...,...,...,...,...
4227760,-94.094,29.693,56.111,9225239.0,0,NaN,615,843
4233738,-26.931,-21.722,83.331,9235196.0,0,NaN,615,844
4228869,-75.539,-41.464,121.789,9226750.0,0,NaN,615,845
4229603,-36.893,-42.331,139.889,9228364.0,0,NaN,615,846


In [66]:
# make sure that we have a good enough proportion of noise to hits
df[df['label'] == 1].shape

(187, 8)

## Generate labels

Once we "explode" the dataset, we need to generate labels. The logic is simple, if `eid1` and `eid2` are same then give it a label of 1, else a label of 0. There are 3 possible combinations that can occur: 1. hit-hit 2. hit-noise and 3. noise-noise. Since noise has `nan` for the event ids and since in Python `nan != nan` all 3 cases can be correctly handled by a simple comparison of the two column values.

In [67]:
df = explode(df)
df

,x1,y1,z1,t1,l1,eid1,ts1,id1,x2,y2,z2,t2,l2,eid2,ts2,id2
4230540,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-36.721,-78.069,94.459,9230328.0,0,NaN,615,1
4230198,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-8.090,87.111,37.789,9229611.0,0,NaN,615,2
4231941,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,2.755,-6.400,65.341,9232477.0,1,2042.0,615,3
4229393,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-75.096,101.731,187.041,9227891.0,0,NaN,615,4
4228900,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-44.772,-21.983,160.189,9226807.0,0,NaN,615,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4229603,-26.931,-21.722,83.331,9235196.0,0.0,NaN,615.0,844.0,-36.893,-42.331,139.889,9228364.0,0,NaN,615,846
4231871,-26.931,-21.722,83.331,9235196.0,0.0,NaN,615.0,844.0,2.734,-6.413,47.411,9232442.0,1,2042.0,615,847
4229603,-75.539,-41.464,121.789,9226750.0,0.0,NaN,615.0,845.0,-36.893,-42.331,139.889,9228364.0,0,NaN,615,846
4231871,-75.539,-41.464,121.789,9226750.0,0.0,NaN,615.0,845.0,2.734,-6.413,47.411,9232442.0,1,2042.0,615,847


In [68]:
df["label"] = df['eid1'] == df['eid2']
df["label"] = df["label"].astype(int)
df

,x1,y1,z1,t1,l1,eid1,ts1,id1,x2,y2,z2,t2,l2,eid2,ts2,id2,label
4230540,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-36.721,-78.069,94.459,9230328.0,0,NaN,615,1,0
4230198,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-8.090,87.111,37.789,9229611.0,0,NaN,615,2,0
4231941,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,2.755,-6.400,65.341,9232477.0,1,2042.0,615,3,0
4229393,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-75.096,101.731,187.041,9227891.0,0,NaN,615,4,0
4228900,88.533,12.246,168.889,9229080.0,0.0,NaN,615.0,0.0,-44.772,-21.983,160.189,9226807.0,0,NaN,615,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4229603,-26.931,-21.722,83.331,9235196.0,0.0,NaN,615.0,844.0,-36.893,-42.331,139.889,9228364.0,0,NaN,615,846,0
4231871,-26.931,-21.722,83.331,9235196.0,0.0,NaN,615.0,844.0,2.734,-6.413,47.411,9232442.0,1,2042.0,615,847,0
4229603,-75.539,-41.464,121.789,9226750.0,0.0,NaN,615.0,845.0,-36.893,-42.331,139.889,9228364.0,0,NaN,615,846,0
4231871,-75.539,-41.464,121.789,9226750.0,0.0,NaN,615.0,845.0,2.734,-6.413,47.411,9232442.0,1,2042.0,615,847,0


In [69]:
df[df['label'] == 1]

,x1,y1,z1,t1,l1,eid1,ts1,id1,x2,y2,z2,t2,l2,eid2,ts2,id2,label
4232560,2.755,-6.400,65.341,9232477.0,1.0,2042.0,615.0,3.0,28.563,-95.175,83.559,9232811.0,1,2042.0,615,6,1
4232171,2.755,-6.400,65.341,9232477.0,1.0,2042.0,615.0,3.0,50.186,-59.163,55.889,9232578.0,1,2042.0,615,11,1
4231652,2.755,-6.400,65.341,9232477.0,1.0,2042.0,615.0,3.0,40.452,-3.427,47.241,9232332.0,1,2042.0,615,34,1
4231939,2.755,-6.400,65.341,9232477.0,1.0,2042.0,615.0,3.0,-8.273,-22.020,47.411,9232477.0,1,2042.0,615,38,1
4231797,2.755,-6.400,65.341,9232477.0,1.0,2042.0,615.0,3.0,50.290,12.112,83.441,9232402.0,1,2042.0,615,47,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231871,50.042,12.160,65.289,9232318.0,1.0,2042.0,615.0,826.0,2.734,-6.413,47.411,9232442.0,1,2042.0,615,847,1
4228383,-36.451,67.187,140.059,9226340.0,1.0,2322.0,615.0,834.0,30.900,86.191,178.511,9226212.0,1,2322.0,615,838,1
4231945,40.122,-3.235,47.241,9232331.0,1.0,2042.0,615.0,835.0,-8.321,-21.772,47.359,9232478.0,1,2042.0,615,839,1
4231871,40.122,-3.235,47.241,9232331.0,1.0,2042.0,615.0,835.0,2.734,-6.413,47.411,9232442.0,1,2042.0,615,847,1


Drop the columns that are not required for training, and write to csv.

In [70]:
df = df.drop(columns=['l1', 'eid1', 'ts1', 'id1', 'l2', 'eid2', 'ts2', 'id2'])
df

,x1,y1,z1,t1,x2,y2,z2,t2,label
4230540,88.533,12.246,168.889,9229080.0,-36.721,-78.069,94.459,9230328.0,0
4230198,88.533,12.246,168.889,9229080.0,-8.090,87.111,37.789,9229611.0,0
4231941,88.533,12.246,168.889,9229080.0,2.755,-6.400,65.341,9232477.0,0
4229393,88.533,12.246,168.889,9229080.0,-75.096,101.731,187.041,9227891.0,0
4228900,88.533,12.246,168.889,9229080.0,-44.772,-21.983,160.189,9226807.0,0
...,...,...,...,...,...,...,...,...,...
4229603,-26.931,-21.722,83.331,9235196.0,-36.893,-42.331,139.889,9228364.0,0
4231871,-26.931,-21.722,83.331,9235196.0,2.734,-6.413,47.411,9232442.0,0
4229603,-75.539,-41.464,121.789,9226750.0,-36.893,-42.331,139.889,9228364.0,0
4231871,-75.539,-41.464,121.789,9226750.0,2.734,-6.413,47.411,9232442.0,0


In [73]:
from sklearn.utils import shuffle

In [74]:
related = df[df['label'] == 1].sample(100)
unrelated = df[df['label'] == 0].sample(100)
sample = pd.concat([related, unrelated])
sample = shuffle(sample)
sample

,x1,y1,z1,t1,x2,y2,z2,t2,label
4228612,-55.636,101.539,130.541,9226390.0,-36.243,101.898,140.111,9226362.0,1
4231676,-36.915,-42.248,38.011,9232632.0,40.431,-3.248,38.011,9232341.0,1
4228100,28.371,-95.175,47.359,9234765.0,-94.528,-6.164,186.931,9226024.0,0
4231211,-36.727,-5.030,121.789,9235980.0,12.389,48.502,47.189,9231592.0,0
4235274,0.807,66.676,65.341,9228364.0,-65.005,85.999,150.959,9238225.0,0
...,...,...,...,...,...,...,...,...,...
4235997,11.206,-94.122,55.889,9233417.0,76.473,31.418,74.211,9239728.0,0
4229165,-94.519,-5.967,160.189,9231682.0,30.670,49.765,121.731,9227420.0,0
4231556,-7.866,-93.377,74.041,9232818.0,50.221,12.181,56.059,9232280.0,1
4232189,28.452,-22.097,55.831,9232455.0,-26.839,-21.775,65.231,9232588.0,1


In [35]:
df.to_csv('../data/slice-615.csv', index=False, header=False)

In [75]:
sample.to_csv('../data/subsample-slice-615.csv', index=False, header=False)

# MLP for "Pattern Matrix" Replacement

In the meantime, we take 10% of timeslice 615 and move forward with training (data/slice-615.csv). I followed this [tutorial](https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/) to implement the first iteration of the network.

In [32]:
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [80]:
class CSVDataset(Dataset):
    def __init__(self, path):
        df = read_csv(path, header=None)
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        self.X = self.X.astype('float32')
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        
    def __len__(self):
        """
        return number of rows in the dataset
        """
        return len(self.X)
    
    def __getitem__(self, idx):
        """
        return a row at specified id
        """
        return [self.X[idx], self.y[idx]]
    
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

class MLP(Module):
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
        
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X
    
def prepare_data(path):
    dataset = CSVDataset(path)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

def train_model(train_dl, model):
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    for epoch in range(100):
        for i, (inputs, targets) in enumerate(train_dl):
#             clear gradients
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
#             update model weights
            optimizer.step()
    
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.round()
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

def predict(row, model):
    pass

In [81]:
path = '../data/subsample-slice-615.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))

134 66


In [82]:
model = MLP(8)
model

MLP(
  (hidden1): Linear(in_features=8, out_features=10, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=10, out_features=8, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=8, out_features=1, bias=True)
  (act3): Sigmoid()
)

In [83]:
train_model(train_dl, model)

In [84]:
acc = evaluate_model(test_dl, model)
print("Accuracy: %.3f" % acc)

Accuracy: 0.515
